In [1]:
import PlottingFunction as pf
import DataManipulation as dm
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from sklearn.cluster import KMeans
import winsound

%matplotlib qt5

In [2]:
df = pd.read_csv(Path.cwd() / 'data' / 'vast' / "vast_serve_200k.csv")
df = dm.keep_data_in_frame(df, compressed=False, only_half=True)
df_deuce = df[df.adpoint==0]
df_ad = df[df.adpoint==1]
df_deuce = dm.change_vast_error_shots(df_deuce,'deuce_serve')
df_ad = dm.change_vast_error_shots(df_ad,'ad_serve')
df_serves = {"Deuce": df_deuce, "Ad": df_ad}


C:\Users\craig\PycharmProjects\TennisResearch - Python\DataManipulation.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.vast)
C:\Users\craig\PycharmProjects\TennisResearch - Python\DataManipulation.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.vast)


In [3]:
means = {}
overlap = False
n_clusters = 3

for df_label in df_serves:
    df = df_serves[df_label]
    if df_label == "Deuce":
        receiver_regions = pf.createPlayerRegions(type = 'deuce_serve')
    else:
        receiver_regions = pf.createPlayerRegions(type = 'ad_serve')
    plt.figure()
    i = 0
    for receiver_region in receiver_regions:
        df_filter = df
        df_filter = dm.filter_data_by_buckets(df_filter, 'receiver_start', receiver_regions[receiver_region], compressed=True)

        if overlap == True:
            cluster = KMeans(n_clusters=n_clusters, random_state=0).fit(df_filter[['x02', 'y02', 'px0', 'py0', 'ox0', 'oy0']])
        else:
            cluster = KMeans(n_clusters=n_clusters, random_state=0).fit(df_filter[['x02', 'y02']])
        centroids = cluster.cluster_centers_
        df_filter["Cluster"] = cluster.labels_
        
        means_tmp = df_filter.groupby('Cluster')['vast'].mean()
        means[df_label, receiver_region] = means_tmp

        plt.subplot(1, 3, 1+i)
        pf.plot1D(df_filter, centroids, means_tmp, alpha_ball = 0.50, receiver_position='receiver_start', just_ball=True, compressed=True, only_half=True, 
                  title =  "Clustered " + df_label + " Serves with the Receiver at the " + receiver_region)
        pf.markupCourt('receiver_start',receiver_regions[receiver_region],compressed=True)
        pf.caption_cluster_plots(df_filter, centroids, n_clusters)
        i=i+1
        
    plt.subplots_adjust(top=0.7,bottom=0.2,left=0.04,right=0.97,hspace=0.15,wspace=0.17)

winsound.Beep(2000, 800)